In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import warnings
warnings.filterwarnings('ignore')

## Merge 3h IMERG data and cut for Veneto Region

In [2]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Italy','Satellite','IMERG','3_hour','yearly')

In [12]:
IMERG_files = glob.glob(os.path.join(dir_base,'*.nc'))
IMERG_files = sorted(IMERG_files)
IMERG_files = IMERG_files[1:] # dont use 2000
IMERG_len = len(IMERG_files)
print(f'Number of IMERG files: {IMERG_len}')

Number of IMERG files: 23


In [13]:
min_lon, max_lon, min_lat, max_lat = 10, 14, 44, 47 # Veneto

In [14]:
PRE = None

for nn in range(IMERG_len):#IMERG_len
    DATA = xr.open_dataset(IMERG_files[nn])
    DATA = DATA.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))
    
    if PRE is None:
        PRE = DATA['PRE'].data
        time = DATA['time'].data
        lat = DATA['lat'].data
        lon = DATA['lon'].data
    
    else:
        pre_tmp = DATA['PRE'].data
        time_tmp = DATA['time'].data
        PRE = np.concatenate([PRE, pre_tmp])
        time = np.concatenate([time, time_tmp])

In [15]:
time = pd.to_datetime(time)

In [16]:
PRE_xr = xr.Dataset(data_vars={"PRE": (("time","lat","lon"), PRE)},
                    coords={'time': time, 'lat': lat, 'lon': lon},
                    attrs=dict(description="IMERG 3 hour data for Veneto region"))

PRE_xr.PRE.attrs["units"] = "mm/3h"
PRE_xr.PRE.attrs["long_name"] = "3 hour precipitation rate (combined microwave-IR) estimate. Formerly precipitationCal"
PRE_xr.PRE.attrs["origname"] = "precipitation"

PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lat.attrs["long_name"] = "Latitude"

PRE_xr.lon.attrs["units"] = "degrees_east"
PRE_xr.lon.attrs["long_name"] = "Longitude"

In [17]:
time_vector = pd.to_datetime(PRE_xr['time'])

yy_s = pd.to_datetime(time_vector[0]).year
mm_s = str(pd.to_datetime(time_vector[0]).month).zfill(2)
dd_s = str(pd.to_datetime(time_vector[0]).day).zfill(2)

yy_e = pd.to_datetime(time_vector[-1]).year
mm_e = str(pd.to_datetime(time_vector[-1]).month).zfill(2)
dd_e = str(pd.to_datetime(time_vector[-1]).day).zfill(2)

print(f'Start time: {dd_s}/{mm_s}/{yy_s}')
print(f'End time: {dd_e}/{mm_e}/{yy_e}')

Start time: 01/01/2001
End time: 31/12/2023


In [18]:
PRE_out = os.path.join('/','media','arturo','Arturo','Data','Italy','Satellite','IMERG','3_hour',f'IMERG_Veneto_3h_{yy_s}_{mm_s}_{dd_s}_{yy_e}_{mm_e}_{dd_e}.nc')
print(f'Export PRE data to {PRE_out}')
PRE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/Arturo/Data/Italy/Satellite/IMERG/3_hour/IMERG_Veneto_3h_2001_01_01_2023_12_31.nc
